In [7]:
from selenium.webdriver import Chrome
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [8]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized");

browser = webdriver.Chrome("chromedriver", options=options)

In [9]:
#서울민원 데이터 
browser.get("https://democracy.seoul.go.kr/front/allSuggest/list.do?sn2=&searchCondition2=&searchCondition3=&searchCondition4=&hashTag=&sKind=M&suggestask_sn=&sDiscussionSn=&sPracticeSn=&sSuggest_divi=&reCom=&reComList_1=%EC%BD%94%EB%A1%9C%EB%82%9819+%EB%B0%94%EC%9D%B4%EB%9F%AC%EC%8A%A4&sKeyword=&pageIndex=1")
browser.implicitly_wait(5)



In [ ]:
articleList = []
dfList = []

for y in range(10):
    for x in range(2,11):
        for i in range(1,13):
            review = browser.find_elements_by_xpath(f'//*[@id="suggList_{i}"]')

            for tmp in review:
                df = pd.DataFrame({
                    "NO":[tmp.find_element_by_xpath('p[2]').text],

                })
            dfList.append(df)

            articleList = pd.concat(dfList)

        browser.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[2]/span/a[{x}]').click()
        browser.implicitly_wait(10)
    browser.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/a[3]').click()
    browser.implicitly_wait(10)

In [ ]:
#데이터 확인
articleList = pd.concat(dfList)

In [ ]:
articleList

In [ ]:
#데이터 처리(한글, 공백 제외)
import re

articleList["NO"] = articleList["NO"].apply(lambda x: re.sub("[^가-힣\s]","",x))
articleList

In [ ]:
#공백 row 삭제

articleList = articleList[articleList["NO"].apply(lambda x: re.sub("[^가-힣]","",x)) !=""]
articleList

In [ ]:
#WordExtractor로 단어 점수 학습

from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(articleList["NO"].values)
words = word_extractor.extract()
words

In [ ]:
#Tokenizer (L-R 분리)

from soynlp.tokenizer import LTokenizer
from soynlp.word import WordExtractor
from soynlp.utils import DoublespaceLineCorpus

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

In [ ]:
articleList["tokenizer"] = articleList["NO"].apply(lambda x: tokenizer.tokenize(x, remove_r=True))
articleList

In [ ]:
#각각의 단어들을 words변수에 append 작업

words = []
for i in articleList["tokenizer"].values:
    for k in i:
        words.append(k)
words

In [ ]:
#각각의 단어들을 counter를 사용하여 빈도수를 확인

from collections import Counter
cnt = Counter(words)
words = dict(cnt)
words

In [ ]:
#내림차순으로 정렬

rank = sorted(words.items(),reverse=True,key=lambda item: item[1])

In [ ]:
#불법주정차 관련 빈도수 분석

for i in range(len(rank)):
    print(i+1,"위: ",rank[i])

In [ ]:
#wordcloud 시각화 진행

from wordcloud import WordCloud
wordcloud = WordCloud(height = 500, width=500,background_color="white")\
                      .generate_from_frequencies(words)

In [ ]:
wordcloud = WordCloud(font_path = 'C:/Windows/Fonts/malgun.ttf', background_color='white', width=500, height=500).generate_from_frequencies(words)

In [ ]:
#wordcloud 시각화

plt.figure(figsize=(10,10))
plt.axis("off")
plt.imshow(wordcloud)
plt.show()